#### as i saw the previous people whom worked in this dataset they achieved high accuracy in their model by using transfer learning techniques as a built in models trained , i figure out it seams like a weapon in their hand  to use when you face underfitting or poor data , its realy impressive, but i don't use it here because i am still learning and not reached the part of pre-trained model yet

# 1.1 Import necessary libraries

In [2]:
pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 991.0 kB/s eta 0:00:39
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   ---------------------------------------- 0.1/38.6 MB 1.1 MB/s eta 0:00:37
   -----------

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import imghdr
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings 
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore')

C:\Users\Sabesh Rajan\AppData\Local\Temp\ipykernel_13528\293857952.py:6: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [ ]:
classes = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_labels = {i:index for index , i in enumerate(classes)}
class_names_labels

# 1.2 checking our data

In [ ]:
train_dir = '../input/intel-image-classification/seg_train/seg_train'
test_dir = '../input/intel-image-classification/seg_test/seg_test'

In [ ]:
def check_extension(exten,data_dir): # exten = "jpg" , data_dir = 'train' or test only >> this func not valid for pred data
    """to check for a valid extensions 
        for train and test dataset """
    for folder in os.listdir(data_dir):
        for image in os.listdir(os.path.join(data_dir,folder)):
            image_path = os.path.join(data_dir,folder,image)
            tip = imghdr.what(image_path)
            if tip not in exten:
                print(f"there is issue with this file |<<'{image_path}'>>|")

extension = ['jpg','jpeg']


In [ ]:
check_extension(extension,train_dir)

In [ ]:
def check_data(data_dir):
    """define shape of the dataset and variety of them"""
    sizes = []
    for folder in os.listdir(data_dir):
        for image in os.listdir(os.path.join(data_dir,folder)):
            image_path = os.path.join(data_dir,folder,image)
            image = cv2.imread(image_path)
            sizes.append(image.shape)
    return pd.Series(sizes).value_counts()

In [ ]:
check_data(train_dir)

# 1.3 Loading Dataset

In [ ]:
h = 150
w = 150
def load_data():
    '''this function is defiend to grap the data from folders of train and test dataset
       and then put all of the data inside a list called output after being converted int RGB COLOR & resized
       to (150,150)'''
    
    data_dir = data_dir = ['../input/intel-image-classification/seg_train/seg_train','../input/intel-image-classification/seg_test/seg_test']
    output = []
    
    for file in data_dir: # iterate over the two type of files {train,test}
        images = [] # to put all images inside it
        labels = [] # to put all corosoponding labels of images inside it
        
        for category in os.listdir(file): # to grap the labels, we have to iterate over all folders 
            label = class_names_labels[category]
            
            for image in os.listdir(os.path.join(file,category)): # iterating over all images
                image_path = os.path.join(file,category,image)   # grap the image path to be able to read it an store it

                img = cv2.imread(image_path)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                img = cv2.resize(img,(h,w))

                labels.append(label)
                images.append(img)

        images = np.array(images,dtype= np.float32)
        labels = np.array(labels,dtype = np.int32)
        output.append((images,labels))
    return output

In [ ]:
((X_train,y_train),(X_test,y_test)) = load_data()

In [ ]:
print(f"x_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")
print(f"x_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")

its important to shuffle our data for modelling

In [ ]:
(X_train,y_train) = shuffle(X_train,y_train,random_state=45)
(X_test,y_test) = shuffle(X_test,y_test,random_state = 45)

# 1.4 Explore our Datset

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i].astype(int))
    plt.xticks()
    plt.yticks()
    plt.title(classes[y_train[i]])
    plt.suptitle('sample of train dataset',fontsize = 20)

# 1.5 Preprocess our Data

In [ ]:
print(f'min value of pixels = {X_train.min()}')
print(f'max value of pixels = {X_train.max()}')

In [ ]:
X_train,x_test = X_train/255.0 , X_test/255.0

# 1.6 Finally building our model

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(h,w,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
# logdir = 'logs'
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
y_test

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer,loss = loss,metrics=['accuracy'])

In [ ]:
y_train= to_categorical(y_train)
y_test= to_categorical(y_test)

In [ ]:
y_test

In [ ]:
# early = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)

hist = model.fit(X_train,y_train,batch_size=32,
                 epochs=30,
                 validation_data=(X_test,y_test),
#                  callbacks=[early],
                 )

### it seems our model has been underfitted the dataset

## lets figure out what has happened during training and validation process

In [ ]:
plt.figure(figsize=(10,5))
loss = hist.history['loss']
val_loss = hist.history['val_loss']
accuracy = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']
epochs = range(len(loss))
plt.subplot(1,2,1)
plt.plot(epochs,loss,label = 'loss')
plt.plot(epochs,val_loss, label = 'val_loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('Loss visualization')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs,accuracy,label = 'accuracy')
plt.plot(epochs,val_accuracy, label = 'val_accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title('accuracy visualization')
plt.legend()

### lets take a tour by taking a sample from pred file to check how far our achievement model reached :)

In [ ]:
pred_dir ='../input/intel-image-classification/seg_pred/seg_pred/'

In [ ]:
pred_images = []
for image in os.listdir(pred_dir):
    image_path = os.path.join(pred_dir,image)
    pred_img = cv2.imread(image_path)
    pred_img = cv2.resize(pred_img,(h,w))
    pred_images.append(pred_img)
pred_images = np.array(pred_images,dtype = np.float32)

In [ ]:
file_array = []

for file in pred_images[:9]:
    files = file/255.0 
    file_array.append(files)
    
file_array = np.array(file_array)

In [ ]:
print(file_array.shape)

In [ ]:
predictions = model.predict(file_array)
predictions.shape

In [ ]:
preds_index = np.argmax(predictions, axis=1)
preds_index

In [ ]:
plt.figure(figsize=(12,9))

for idx, i in enumerate(preds_index):
    plt.subplot(3,3, idx+1)
    plt.imshow(file_array[idx])
    plt.title(f"prediction of image: {classes[i]}")
    plt.xticks([])
    plt.yticks([])
    
plt.suptitle("Samples predictions of model on unseen data", size=20, weight='bold')
plt.show()